In [213]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import urlopen
import requests



In [214]:
html = 'https://en.wikipedia.org/wiki/Special_wards_of_Tokyo'
result = requests.get(html).text
soup = BeautifulSoup(result,'xml')
table = soup.find('table',{'class':'wikitable sortable'})

In [347]:
column_name = ['Name','Density','Population','Major_district']
df = pd.DataFrame(columns = column_name)

In [380]:
name=[]
pop = []
density = []
major_dist = []

for row in table.findAll('tr'):
    Ward = row.findAll('td')
    if len(Ward)==8: 
        name.append(Ward[2])
        pop.append(Ward[4])
        density.append(Ward[5].find(text=True))
        major_dist.append(Ward[7].find(text=True))

density[0] = '5100'
new_density = []
for i in range(len(density)):
    x=density[i].replace(',','')
    new_density.append(x)

new_density = [int(x) for x in new_density]


new_name = []
for i in range(len(name)):
    names = name[i].findAll('a')
    new_name.append(names)

new_name = [item for sublist in new_name for item in sublist]
#print (new_name)

name1 = []
for i in new_name:
    name1.append(i.get('title'))
#print (name1)
name2 = []

replace = {'Chiyoda, Tokyo':'Chiyoda', 'Chūō, Tokyo':'Chuo', 'Minato, Tokyo':'Minato', 
            'Sumida, Tokyo':'Sumida', 'Koto, Tokyo':'Koto', 'Ōta, Tokyo':'Ota', 
            'Nakano, Tokyo':'Nakano', 'Kita, Tokyo':'Kita', 'Arakawa, Tokyo':'Arakawa', 
            'Adachi, Tokyo':'Adachi', 'Edogawa, Tokyo':'Edogawa'}

name2 = [replace.get(n,n) for n in name1]
#print(name2)

#print(pop)
new_pop = []
for i in range(len(pop)):
    #print(pop[i])
    pops = pop[i].text[1:9]
    new_pop.append(pops)
#print(new_pop)
new_population = []
for i in range(len(new_pop)):
    x = new_pop[i].replace(',','')
    new_population.append(x)
new_population = [int(x) for x in new_population]

replace_districts = {'Nagatachō':'Nagatacho', 'Hongō':'Hongo', 'Kinshichō':'Kinshicho', 'Ōmori': 'Omori', 
                     'Kōenji':'Koenji', 'Arakawa, Machiya,':'Arakawa', 'Ayase,':'Ayase', 'Kasai, Koiwa\n':'Kasai'}
major_district = []

major_district = [replace_districts.get(n,n) for n in major_dist]

#make dataframe
df['Name'] = name2
df['Density'] = new_density
df['Population'] = new_population
df['Major_district'] = major_district

df.head()

,Name,Density,Population,Major_district
0,Chiyoda,5100,59441,Nagatacho
1,Chuo,14460,147620,Nihonbashi
2,Minato,12180,248071,Odaiba
3,Shinjuku,18620,339211,Shinjuku
4,Bunkyō,19790,223389,Hongo


In [382]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Tokyo,Japan'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

geo_df=pd.read_csv('tokyo_coord.csv')
geo_df = geo_df[geo_df.columns[0:3]]
#geo_df.head()

Tokyo = pd.merge(df,geo_df,on='Name')
Tokyo.head()

,Name,Density,Population,Major_district,Latitude,Longitude
0,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469
1,Chuo,14460,147620,Nihonbashi,35.684058,139.774501
2,Minato,12180,248071,Odaiba,35.619128,139.779403
3,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632
4,Bunkyō,19790,223389,Hongo,35.708800,139.760100


In [ ]:
#wo have all the information of the Tokyo specific wards here (lati, long of each ward, the population.), we can do the analysis
# on those data to define which kinds of restaurant should I open and what is the difference of the venue between Tokyo and Toronto